In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [2]:
CSV_files ={"customers": r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_customers_dataset.csv",
"geolocation_dataset":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_geolocation_dataset.csv",
"order_items":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_order_items_dataset.csv",
"order_payments":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_order_payments_dataset.csv",
"order_review":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_order_reviews_dataset.csv",
"order_dataset":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_orders_dataset.csv",
"list_product":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_products_dataset.csv",
"list_seller":r"C:\Users\mzanola\project\.venv\final_project\dataset\olist_sellers_dataset.csv",
"product_category":r"C:\Users\mzanola\project\.venv\final_project\dataset\product_category_name_translation.csv"}

In [3]:
dataframes = {name: pd.read_csv(path) for name, path in CSV_files.items()}

In [4]:
df_orders_items = dataframes["order_items"]
df_order_payments = dataframes["order_payments"]
df_order_review= dataframes["order_review"]
df_order_dataset = dataframes["order_dataset"]
df_list_product = dataframes["list_product"]
df_list_seller = dataframes["list_seller"]
df_product_category = dataframes["product_category"]
df_geolocation_dataset = dataframes["geolocation_dataset"]